In [14]:
import sys
import os
sys.path.append('../estimator')
from classifiers import ActionClassifier

## Normalize training data

In [ ]:
# Normalize training data

## Train Action Classifier with normalized training data

In [ ]:
# Train action classifier with normalized training data

## Normalize and predict test actions with trained Action Classifier

In [ ]:
# Normalize and predict test values with trained action classifier

# DAB

In [ ]:
print("dab")

## Normalize training data

In [15]:
# Normalize training data

## Train Action Classifier with normalized training data

In [16]:
# Train action classifier with normalized training data

## Normalize and predict test actions with trained Action Classifier

In [17]:
# Normalize and predict test values with trained action classifier

# DAB

In [18]:
print("dab")

dab
